$S$

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas

In [2]:
NL_USERS = {}
TWITTER_USERS = {}

In [164]:
paginated_no =0
nl_url = 'https://www.nairaland.com/5998585/bbnaija-2020-live-updates-thread/' + str(paginated_no)

In [165]:
str(nl_url)

'https://www.nairaland.com/5998585/bbnaija-2020-live-updates-thread/0'

In [167]:
nl_scrape = requests.get(str(nl_url))

In [168]:
nl_scrape = requests.get(str(nl_url))
nl_soup = bs(nl_scrape.content, "html.parser")

In [169]:
nl_soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"
"http://www.w3.org/TR/html4/loose.dtd">
<html><head><title>BBNaija 2020 Live Updates Thread - TV/Movies - Nigeria</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><link href="https://www.nairaland.com/static/nl2.css/938989384213901" rel="stylesheet" type="text/css"/><meta content="KUABan1rWBhK3Jb2LePxHAv00FEbbM1wQMAAgsN-EyA" name="google-site-verification"/><link href="/5998585/bbnaija-2020-live-updates-thread/1" rel="next"/></head><body><div class="body"><table id="up" summary=""><tr><td class="grad"><h1><a class="g" href="https://www.nairaland.com/" title="Nairaland, the Nigerian Forum"><img height="24" src="/static/logo1.png" width="246"/></a></h1> Welcome, <b>Guest</b>: <a href="/register">Join Nairaland</a> / <b><a href="/login">LOGIN!</a></b> / <a href="/trending">Trending</a> / <a href="/recent">Recent</a> / <a href="/topics">New</a><br/><b>Stats: </b>2,517,806 members, 5,750,582 topics. <b>

In [170]:
"""
<a class="user" href="/vizkiz" title="Location: Lagos Nigeria">vizkiz</a>
<span class="s"><b>12:01am</b> On <b>Jul 18</b></span>
<tr><td class="l w pd" id="pb91825753"><div class="narrow"><i
"""

'\n<a class="user" href="/vizkiz" title="Location: Lagos Nigeria">vizkiz</a>\n<span class="s"><b>12:01am</b> On <b>Jul 18</b></span>\n<tr><td class="l w pd" id="pb91825753"><div class="narrow"><i\n'

In [171]:
users_ = nl_soup.findAll("a", {"class":"user"}, recursive=True)
posts_ = nl_soup.findAll("div", {"class":"narrow"}, recursive=True)
timestamps_ = nl_soup.findAll("span", {"class":"s"}, recursive=True)

In [11]:
dl_name = []
dl_loc = []

In [12]:
import re

In [13]:
def clean_text(text):
    """Remove html tags from a string and fetch the text"""
    clean = re.compile('<.*?>')
    new_clean = re.sub(clean, '', text)
    text = " ".join(re.findall("[A-Za-z]+", new_clean))
    return text

In [81]:
import time
import csv
from datetime import datetime

In [82]:
def log_error(page_link, error_status):
    today, now = datetime.today(), datetime.now()

    #create directory for storing logs for the day
    directory = 'errorlog_'+str(today.day)+'_'+str(today.strftime("%b")).lower()+'_'+str(today.year)
    if not os.path.exists(directory):
        os.makedirs(directory)

    logged_link = str(page_link)
    error_type = str(error_status)
    error_csv = [str(today), str(now.hour)+':'+str(now.minute)+':'+str(now.second),str(logged_link),str(error_type)]
    error_file = open(str(directory)+'/log_error_'+str(directory)+'.csv', 'a')
    with error_file:
        writer = csv.writer(error_file)
        writer.writerow(error_csv)
    return

In [123]:
from datetime import datetime
import csv
import json
import os

def output_file(domain, user_details):
    print("Exporting your products to file")
    today = datetime.now()

    #creates folder to save scraped data for today
    directory = 'Task_'+str(today.day)+'_'+str(today.strftime("%b")).lower()+'_'+str(today.year)
    if not os.path.exists(directory):
        os.makedirs(directory)

    output_file_json = str(directory)+'/'+str(domain) + '_details_'+str(directory)+'.json'
    with open(output_file_json, 'w') as outfile:
        json.dump(user_details, outfile)
    return

In [219]:
scraped_pages = []
user_details = []
domain_name = "https://www.nairaland.com/5998585/bbnaija-2020-live-updates-thread/"


def timeout(t_in_s):
    #timeout for provided time(seconds) + 5 seconds more
    time.sleep(int(t_in_s) + 5)
    

def Nl_Ban_Scraper(page_link, paginated_no=None):
    if paginated_no:
        nl_url = str(page_link)+str(paginated_no)
    nl_url = str(page_link)
    timeout(5) #chill for 10s
    
    if nl_url not in scraped_pages:
        scraped_pages.append(nl_url)
        nl_scrape = requests.get(str(nl_url), headers=spook_header())
        nl_soup = bs(nl_scrape.content, "html.parser")
        users_ = nl_soup.findAll("a", {"class":"user"}, recursive=True)
        posts_ = nl_soup.findAll("div", {"class":"narrow"}, recursive=True)
        timestamps_ = nl_soup.findAll("span", {"class":"s"}, recursive=True)
        for usr_,post_,datetime_ in zip(users_, posts_, timestamps_):
        #fetch user details('username', 'datetime', and 'location')
            try:
                datetime = datetime_.text.split(" ")
                time_, date_= str(datetime[0]), str(datetime[2] + datetime[3])
                if usr_.text:
                    username = str(usr_.text)
                else:
                    username = "User Deleted"
                
                post = _clean_post(post_)
                location = str(usr_["title"].split(" ")[1:])
            
            except KeyError:
                location = "No location"
#                 continue
            except Exception:
                pass

            finally:
                user_details.append({
                    'username': username,
                    'location': clean_text(location),
                    'post': post,
                    'time': time_,
                    'date': date_
                    })
                

        #Handle pagination (Nairaland doesn't have a dedicated next button to do something like)
        #pageNextLink instead we increment page number
        output_file("NL", user_details)
        users_, posts_, timestamps_ = [], [], [] #empty users, posts and timestamps
        _paginated_no = len(scraped_pages)
        next_url_to_scrape = str(domain_name) + str(_paginated_no)
        timeout(55) #chill for 60s
        Nl_Ban_Scraper(next_url_to_scrape)
        
    else:
        #log issues(missing/invalid/already scraped pagelink)
        log_error(page_link, 'Pagination issues')
        output_file("domain", user_details)
        timeout(3)
        return
    return

In [220]:
Nl_Ban_Scraper("https://www.nairaland.com/5998585/bbnaija-2020-live-updates-thread/")

Exporting your products to file


KeyboardInterrupt: 

In [221]:
print(len(user_details))
user_details

32


[{'username': 'vizkiz',
  'location': 'Lagos Nigeria',
  'post': 'quote detected',
  'time': '12:01am',
  'date': 'Jul18'},
 {'username': 'vizkiz',
  'location': 'Lagos Nigeria',
  'post': 'quote detected',
  'time': '12:02am',
  'date': 'Jul18'},
 {'username': 'homesteady',
  'location': 'from kano to kaduna to benin',
  'post': 'possible spam detected',
  'time': '12:03am',
  'date': 'Jul18'},
 {'username': 'vizkiz',
  'location': 'Lagos Nigeria',
  'post': 'possible spam detected',
  'time': '12:05am',
  'date': 'Jul18'},
 {'username': 'kodded',
  'location': 'color b i when will nairaland bring in the b i color color color color Dislike color color color color b i button b i color',
  'post': 'possible spam detected',
  'time': '12:07am',
  'date': 'Jul18'},
 {'username': 'KamsiViva',
  'location': 'No location',
  'post': 'The show never start ona don dey rant It s alright let me be chewing my suya first until they post my ex as one of the contestants',
  'time': '12:09am',
  'dat

In [69]:
len(user_details)

32

In [212]:
def _clean_post(post):
    spam = ["call", "whatsapp", "watsapp", "<img>"]
    try:
        if "<blockquote>" in str(post):
            post = "quote detected"
        elif "://" in str(post) or "www." in str(post) or str(post).lower() in spam:
            post = "possible spam detected"
        elif post.text == '':
            post = "POST DELETED BY MOD"
        else:
            post = str(post.text)
    except Exception:
        pass
    else:
        clean_post_ = clean_text(post)
    return clean_post_

In [211]:
_clean_post(posts_[27])

'Hello everyone'

In [182]:
clean_text("fff")

'fff'

In [20]:
import requests
import random 
from collections import OrderedDict

def spook_header():
    headers_list = [
        # Firefox 77 Mac
         {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            "Accept-Language": "en-US,en;q=0.5",
            "Referer": "https://www.google.com/",
            "DNT": "1",
            "Connection": "keep-alive",
            "Upgrade-Insecure-Requests": "1"
        },
        # Firefox 77 Windows
        {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip, deflate, br",
            "Referer": "https://www.google.com/",
            "DNT": "1",
            "Connection": "keep-alive",
            "Upgrade-Insecure-Requests": "1"
        },
        # Chrome 83 Mac
        {
            "Connection": "keep-alive",
            "DNT": "1",
            "Upgrade-Insecure-Requests": "1",
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
            "Sec-Fetch-Site": "none",
            "Sec-Fetch-Mode": "navigate",
            "Sec-Fetch-Dest": "document",
            "Referer": "https://www.google.com/",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
        },
        # Chrome 83 Windows 
        {
            "Connection": "keep-alive",
            "Upgrade-Insecure-Requests": "1",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
            "Sec-Fetch-Site": "same-origin",
            "Sec-Fetch-Mode": "navigate",
            "Sec-Fetch-User": "?1",
            "Sec-Fetch-Dest": "document",
            "Referer": "https://www.google.com/",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept-Language": "en-US,en;q=0.9"
        }
    ]

    return random.choice(headers_list)

In [21]:
spook_header()

{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
 'Accept-Language': 'en-US,en;q=0.5',
 'Referer': 'https://www.google.com/',
 'DNT': '1',
 'Connection': 'keep-alive',
 'Upgrade-Insecure-Requests': '1'}

In [498]:
c = "https://www.nairaland.com/5998585/bbnaija-2020-live-updates-thread/600"

In [499]:
c[-1]

'0'

In [440]:
xxx = []
for i in nl_soup.findAll("div", {"class":"narrow"}, recursive=True):
    if "<blockquote>" in str(i):
        print("quote detected")
        xxx.append("quote")
        continue
    if "://" in str(i) or "www." in str(i):
        print("possible spam detected")
        xxx.append("spam")
        continue
    else:
        xxx.append(clean_text(str(i.text)))
        print(i.text)

quote detected
quote detected
possible spam detected
possible spam detected
possible spam detected
The show never start ona don dey rant...It's alright, let me be chewing my suya first, until they post my ex as one of the contestants.
possible spam detected
possible spam detected
2 days to go
quote detected
quote detected
I heard one descendant of bobrisky is going to the house.... "Jay boogie" as a male or female??
possible spam detected
possible spam detected
Another season of IMMORAL NAIJA SHOW. And people will foolishly waste their money and airtime voting
Following...
After all the gra gra of the first day, we will come to start talking business...
Team female winner.
Watching from Nairaland.
quote detected
quote detected
It's a wrap for Mercy, Tacha and Co. Life
This troll is back again? Only you will open BBNaija threads and still troll them to ensure they reach a high view count. I’m sure that Yoruba boy pays you for the effort. This time I’ll focus my mind on ways to get money

In [462]:
[word for sentence in text for word in sentence]

32

In [467]:
# x = [t.text for t in timestamps_]

In [472]:
x = [t.text for t in timestamps_]
xc = [i.split(" ") for i in x]

In [477]:
for i in x:
    print(i.split(" "))

['12:01am', 'On', 'Jul', '18']
['12:02am', 'On', 'Jul', '18']
['12:03am', 'On', 'Jul', '18']
['12:05am', 'On', 'Jul', '18']
['12:07am', 'On', 'Jul', '18']
['12:09am', 'On', 'Jul', '18']
['12:27am', 'On', 'Jul', '18']
['12:27am', 'On', 'Jul', '18']
['12:28am', 'On', 'Jul', '18']
['12:32am', 'On', 'Jul', '18']
['12:33am', 'On', 'Jul', '18']
['12:41am', 'On', 'Jul', '18']
['12:47am', 'On', 'Jul', '18']
['12:53am', 'On', 'Jul', '18']
['12:54am', 'On', 'Jul', '18']
['1:09am', 'On', 'Jul', '18']
['1:22am', 'On', 'Jul', '18']
['1:50am', 'On', 'Jul', '18']
['4:08am', 'On', 'Jul', '18']
['4:12am', 'On', 'Jul', '18']
['5:28am', 'On', 'Jul', '18']
['5:29am', 'On', 'Jul', '18']
['6:17am', 'On', 'Jul', '18']
['7:15am', 'On', 'Jul', '18']
['9:16am', 'On', 'Jul', '18']
['9:17am', 'On', 'Jul', '18']
['9:28am', 'On', 'Jul', '18']
['9:28am', 'On', 'Jul', '18']
['9:31am', 'On', 'Jul', '18']
['9:32am', 'On', 'Jul', '18']
['9:54am', 'On', 'Jul', '18']
['10:13am', 'On', 'Jul', '18']


In [86]:
print(f"fff")

fff


In [240]:
def contestant_points(post):
    nengi_score = 0
    if "nengi" in post:
        nengi_score += 1
    return nengi_score

In [246]:
contestant_points("ffff, nengi")

1

In [250]:
# [1 if 'Love3' in "we Love" else 0]
p = "Sdndkd dfew Dkein4 mfe"
p.lower()

'sdndkd dfew dkein4 mfe'

In [251]:
def sent_by_contestant():
    post_ = post.lower()
    result = Analyzer(post_)
    pos_neu_neg = process_result()
    if "nengi" in post_:
        sent_Nenji = score
    else:
        sent_Nenji = 0
    if "prince" in post_:
        sent_Prince = score
    else:
        sent_Prince = 0
    if "erica" in post_:
        sent_Erica = score
    else:
        sent_Erica = 0
    if "dorathy" in post_ or "doro" in post_:
        sent_Dorathy = 1
    else:
        sent_Dorathy = 0
    if "Tolanibaj" in post_ or :
        sent_Nenji = 1
    else:
        sent_Nenji = 0
    if "vee" in post_:
        sent_Nenji = 1
    else:
        sent_Nenji = 0
    if "lilo" in post_:
        sent_Nenji = 1
    else:
        sent_Nenji = 0
    if "trickytee" in post_ or "tee" in post_:
        sent_Tee = 1
    else:
        sent_Tee = 0
    if "lucy" in post_:
        sent_Lucy = 1
    else:
        sent_Lucy = 0
    if "kiddwaya" in post_ or "kidd" in post_:
        sent_Kiddwaya = 1
    else:
        sent_Kiddwaya = 0
    if "praise" in post_:
        sent_Praise = 1
    else:
        sent_Praise = 0
    if "wathoni" in post_:
        sent_Wathoni = 1
    else:
        sent_Wathoni = 0
    if "eric" in post_:
        sent_Eric = 1
    else:
        sent_Eric = 0
    if "kaisha" in post_ or "aisha" in post:
        sent_Kaisha = 1
    else:
        sent_Kaisha = 0
    if "neo" in post_:
        sent_Neo = 1
    else:
        sent_Neo = 0
    if "brighto" in post_ or "bright" in post_:
        sent_Brighto = 1
    else:
        sent_Brighto = 0
    if "laycon" in post_ or "lekan" in post_:
        sent_Laycon = 1
    else:
        sent_Laycon = 0
    if "tochi" in post_ or "tochukwu" in post_:
        sent_Tochi = 1
    else:
        sent_Tochi = 0
    if "ka3na" in post_ or "katrina" in post_:
        sent_Ka3na = 1
    else:
        sent_Ka3na = 0


In [253]:
vv = [{"username": "vizkiz", "location": "Lagos Nigeria", "post": "quote detected", "time": "12:01am", "date": "Jul18"}, {"username": "vizkiz", "location": "Lagos Nigeria", "post": "quote detected", "time": "12:02am", "date": "Jul18"}]

In [255]:
for i in vv:
    print(i["username"])

vizkiz
vizkiz
